# Data Integration & Preprocessing for Customer Satisfaction Analysis

This notebook is part of a larger project exploring customer satisfaction in Brazilian e-commerce using the [Olist dataset](https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce).  
It builds on the cleaned datasets prepared in the previous notebook by joining them into a single analytical table, performing additional data cleaning, and engineering features to support the upcoming analysis.


**Goals of this notebook:**

- Join the cleaned tables into a consolidated dataset at the order level  
- Clean and standardize the merged dataset (e.g., data types, missing values)  
- Create new features to capture relevant aspects of orders, products, payments etc.
- Export the final dataset for analysis

**This notebook is preceded and followed by:**

- [Data Cleaning Notebook](./01_data_cleaning.ipynb): loads and prepares the individual raw datasets for integration  
- [Exploratory Analysis Notebook](./03_customer-satisfaction-analysis.ipynb): investigates customer satisfaction patterns and key influencing factors
---

## **Structure of the Notebook**

> _Note: Section links and “Back to top” links work best in Jupyter environments (e.g., Jupyter Lab or VS Code). They may not work as expected when clicked directly on GitHub._

- [Data Integration](#data-integration)
  - [Loading cleaned datasets](#loading-cleaned-csv-files-into-duckdb-tables)
  - [Joining tables into order-level dataset](#joining-tables-into-order-level-dataset)

In [1]:
import duckdb
import os
import sys
sys.path.append(os.path.abspath(".."))
import utils as u
import pandas as pd

## **Data Integration**

### Loading Cleaned CSV Files into DuckDB Tables

In [2]:
# Path to the folder containing cleaned CSV files
data_path = "../data/cleaned"

# Connect to an in-memory DuckDB database
con = duckdb.connect(database=":memory:")

# Load each cleaned dataset into its own DuckDB table
con.execute(f"""
    CREATE TABLE customers AS 
    SELECT * FROM read_csv_auto('{data_path}/customers.csv');
""")

con.execute(f"""
    CREATE TABLE orders AS 
    SELECT * FROM read_csv_auto('{data_path}/orders.csv');
""")

con.execute(f"""
    CREATE TABLE items AS 
    SELECT * FROM read_csv_auto('{data_path}/items.csv');
""")

con.execute(f"""
    CREATE TABLE payments AS 
    SELECT * FROM read_csv_auto('{data_path}/payments.csv');
""")

con.execute(f"""
    CREATE TABLE reviews AS 
    SELECT * FROM read_csv_auto('{data_path}/reviews.csv');
""")

con.execute(f"""
    CREATE TABLE products AS 
    SELECT * FROM read_csv_auto('{data_path}/products.csv');
""")

con.execute(f"""
    CREATE TABLE sellers AS 
    SELECT * FROM read_csv_auto('{data_path}/sellers.csv');
""")

[🠉 Back to top](#structure-of-the-notebook)

### Joining Tables into Order-Level Dataset

To understand what drives **customer satisfaction**, we need a dataset that captures the entire customer experience — including products purchased, delivery timing, seller location, payment methods, and reviews.  
Since **reviews** are provided **for whole orders** rather than individual products, all features must be aggregated or merged to the **order level** to allow meaningful analysis.

To build this dataset, we:

- **Join** cleaned transactional tables — `items`, `products`, `sellers`, `orders`, `payments`, `reviews`, and `customers` — to gather all relevant data per order  
- **Aggregate** product-level data to the order level (e.g., summing prices, averaging product dimensions)
- **Capture variability** in product categories, sellers, and states:  
  - Orders with more than one product category are labeled `"multiple_categories"`  
  - Similarly, `"multiple_sellers"` and `"multiple_states"` indicate mixed-origin orders  
- **Summarize payment behavior** by counting payment records and installments per order  
- **Calculate time-based features** such as shipping and delivery delays
- **Filter** the data to include only **delivered** orders with a **non-null review score**, ensuring we focus only on completed and rated transactions

This ensures that **each row corresponds to a unique order** and contains a rich set of features to analyze customer satisfaction drivers.

<details>
<summary><strong>How the Order-Level Dataset Was Built (Click to expand)</strong></summary>


**➤ `item_quantities` (Per-Product-Per-Order)**

- Combines `items`, `products`, and `sellers` to generate one row per product within each order.
- Handles nulls using `COALESCE` to label missing product categories or seller info as `'unknown'`.
- Aggregates:
  - `COUNT(*)` for number of times a product was purchased (product quantity)
  - `SUM(price)` and `SUM(freight_value)`
  - `MAX(product_weight_g)` and `MAX(shipping_limit_date)` for key logistics indicators
- Pulls product-level features using `AVG()` (later rounded) for:
  - `product_name_length`, `product_description_length`
  - `product_photos_qty`, `product_length_cm`, `product_height_cm`, `product_width_cm`

---

**➤ Product and Seller Categorization**

**`category_per_order`**
- If all items in an order share the same category → keep it.
- If there are multiple → label as `'multiple_categories'`.

**`seller_state_per_order` and `seller_id_per_order`**
- Same logic: return actual seller or state if unique, otherwise label as `'multiple_sellers'` or `'multiple_states'`.

**`product_features_per_order`**
- Averages and rounds product features at the order level to summarize size, content richness, and complexity of the products in the order.

---

**➤ `orders_items` (Aggregate Order Metrics)**

- Aggregates order content:
  - Number of **unique products** and **total quantity**
  - **Total price**, **freight**, and **total amount**
  - **Total order weight**
  - Shipping deadline

---

**➤ `payments_agg` (Payment Profile)**

- Aggregates payment behavior:
  - Max installments
  - Number of payment records
  - Aggregated distinct payment types (as a comma-separated string)

---

**➤ Final Output**

Combines all previous aggregations and adds:

- Review score and timing info from `reviews`
- Actual vs. estimated delivery dates from `orders`
- Customer and seller metadata
- Product-level feature summaries

Also includes calculated columns:

- `freight_share`, `freight_to_price_ratio`
- `shipping_delay_days`, `delivery_delay_days`, `review_response_delay_days`

---

**➤ Final Filtering**

Only includes:
- Orders marked as **'delivered'**
- Orders that **have a review score**

</details>

In [3]:
# Build Query to Create Order-Level Dataset
q_orders = """
WITH item_quantities AS (
  -- Aggregate product and seller info per product in each order
  SELECT
    i.order_id,
    i.product_id,
    COALESCE(p.product_category_name_english, 'unknown') AS product_category_name,
    COALESCE(s.seller_state, 'unknown') AS seller_state,
    COALESCE(s.seller_id, 'unknown') AS seller_id,
    COUNT(*) AS product_quantity,
    SUM(i.price) AS product_price,
    SUM(i.freight_value) AS product_freight,
    MAX(p.product_weight_g) AS product_weight_g,
    MAX(i.shipping_limit_date) AS shipping_limit_date,
    AVG(p.product_name_lenght) AS product_name_length,
    AVG(p.product_description_lenght) AS product_description_length,
    AVG(p.product_photos_qty) AS product_photos_qty,
    AVG(p.product_length_cm) AS product_length_cm,
    AVG(p.product_height_cm) AS product_height_cm,
    AVG(p.product_width_cm) AS product_width_cm
  FROM items AS i
  LEFT JOIN products AS p ON i.product_id = p.product_id
  LEFT JOIN sellers AS s ON i.seller_id = s.seller_id
  GROUP BY i.order_id, i.product_id, s.seller_state, s.seller_id, p.product_category_name_english
),

-- Collapse product category into a single value per order
category_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT product_category_name) = 1 
        THEN MAX(product_category_name)
      ELSE 'multiple_categories'
    END AS product_category_name
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse seller state into a single value per order
seller_state_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT seller_state) = 1 
        THEN MAX(seller_state)
      ELSE 'multiple_states'
    END AS seller_state
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse seller ID into a single value per order
seller_id_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT seller_id) = 1 
        THEN MAX(seller_id)
      ELSE 'multiple_sellers'
    END AS seller_id
  FROM item_quantities
  GROUP BY order_id
),

-- Collapse product ID into a single value per order
product_id_per_order AS (
  SELECT
    order_id,
    CASE 
      WHEN COUNT(DISTINCT product_id) = 1 
        THEN MAX(product_id)
      ELSE 'multiple_products'
    END AS product_id
  FROM item_quantities
  GROUP BY order_id
),

-- Average product-level features across items in the order
product_features_per_order AS (
  SELECT
    order_id,
    ROUND(AVG(product_name_length)) AS product_name_length,
    ROUND(AVG(product_description_length)) AS product_description_length,
    ROUND(AVG(product_photos_qty)) AS product_photos_qty,
    ROUND(AVG(product_length_cm)) AS product_length_cm,
    ROUND(AVG(product_height_cm)) AS product_height_cm,
    ROUND(AVG(product_width_cm)) AS product_width_cm
  FROM item_quantities
  GROUP BY order_id
),

-- Aggregate item-level purchase details per order
orders_items AS (
  SELECT
    order_id,
    COUNT(DISTINCT product_id) AS num_unique_products,
    ROUND(SUM(product_quantity)) AS num_items,
    SUM(product_price) AS total_price,
    SUM(product_freight) AS total_freight,
    SUM(product_price + product_freight) AS total_amount,
    SUM(product_quantity * product_weight_g) AS total_order_weight,
    MAX(shipping_limit_date) AS shipping_limit_date
  FROM item_quantities
  GROUP BY order_id
),

-- Aggregate payment details per order
payments_agg AS (
  SELECT
    order_id,
    MAX(payment_installments) AS max_payment_installments,
    COUNT(payment_sequential) AS n_payment_records, 
    STRING_AGG(DISTINCT payment_type, ', ' ORDER BY payment_type) AS payment_types,
    COUNT(DISTINCT payment_type) AS n_payment_types
  FROM payments
  GROUP BY order_id
)

------ Final Joined Order-Level Table ------
SELECT
  oi.order_id,
  r.review_score,
  pid.product_id,
  cat.product_category_name,
  oi.num_unique_products,
  oi.num_items,
  oi.total_price,
  oi.total_freight,
  oi.total_amount,
  oi.total_order_weight,
  ROUND(oi.total_freight / NULLIF(oi.total_amount, 0), 4) AS freight_share,
  ROUND(oi.total_freight / NULLIF(oi.total_price, 0), 4) AS freight_to_price_ratio,
  pf.product_name_length,
  pf.product_description_length,
  pf.product_photos_qty,
  pf.product_length_cm,
  pf.product_height_cm,
  pf.product_width_cm,
  pa.payment_types,
  pa.n_payment_types,
  pa.max_payment_installments,
  pa.n_payment_records,
  oi.shipping_limit_date,
  o.order_delivered_carrier_date,

  -- Calculate shipping delay (days late shipping vs. promised date)
  CASE 
    WHEN o.order_delivered_carrier_date IS NOT NULL AND oi.shipping_limit_date IS NOT NULL 
    THEN DATE_PART('day', o.order_delivered_carrier_date - oi.shipping_limit_date)
    ELSE NULL 
  END AS shipping_delay_days,
  o.order_delivered_customer_date,
  o.order_estimated_delivery_date,

  -- Calculate delivery delay (days late vs. estimated)
  CASE 
    WHEN o.order_delivered_customer_date IS NOT NULL AND o.order_estimated_delivery_date IS NOT NULL 
    THEN DATE_PART('day', o.order_delivered_customer_date - o.order_estimated_delivery_date) 
    ELSE NULL 
  END AS delivery_delay_days,
  r.review_comment_message,
  r.review_creation_date,
  r.review_answer_timestamp,

  -- Time between review creation and publication (platform-side delay)
  CASE 
    WHEN r.review_answer_timestamp IS NOT NULL AND r.review_creation_date IS NOT NULL 
    THEN DATE_PART('day', r.review_answer_timestamp - r.review_creation_date) 
    ELSE NULL 
  END AS review_processing_delay_days,
  
  c.customer_unique_id,
  c.customer_state,
  ssoid.seller_id,
  sso.seller_state

FROM orders_items AS oi
LEFT JOIN orders AS o USING(order_id)
LEFT JOIN customers AS c USING(customer_id)
LEFT JOIN payments_agg AS pa USING(order_id)
LEFT JOIN reviews AS r USING(order_id)
LEFT JOIN category_per_order AS cat USING(order_id)
LEFT JOIN seller_state_per_order AS sso USING(order_id)
LEFT JOIN seller_id_per_order AS ssoid USING(order_id)
LEFT JOIN product_features_per_order AS pf USING(order_id)
LEFT JOIN product_id_per_order AS pid USING(order_id)

-- Filter: only delivered orders with valid review scores
WHERE o.order_status = 'delivered' 
  AND r.review_score IS NOT NULL

ORDER BY oi.order_id;
"""

In [4]:
# Execute the final SQL query and convert the result to a pandas DataFrame
df_orders = con.execute(q_orders).df()

# Show shape of the resulting dataset
print(f"Order-level joined table has {df_orders.shape[0]} rows and {df_orders.shape[1]} columns.")

# Preview the first few rows
df_orders.head()

Order-level joined table has 94782 rows and 36 columns.


,order_id,review_score,product_id,product_category_name,num_unique_products,num_items,total_price,total_freight,total_amount,total_order_weight,...,order_estimated_delivery_date,delivery_delay_days,review_comment_message,review_creation_date,review_answer_timestamp,review_processing_delay_days,customer_unique_id,customer_state,seller_id,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,5,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,1,1.0,58.90,13.29,72.19,650.0,...,2017-09-29,-8,"Perfeito, produto entregue antes do combinado.",2017-09-21,2017-09-22 10:57:03,1,871766c5855e863f6eccc05f988b23cb,RJ,48436dade18ac8b2bce089ec2a041202,SP
1,00018f77f2f0320c557190d7a144bdd3,4,e5f2d52b802189ee658865ca93d83a8f,pet_shop,1,1.0,239.90,19.93,259.83,30000.0,...,2017-05-15,-2,None,2017-05-13,2017-05-15 11:34:13,2,eb28e67c4c0b83846050ddfb8a35d051,SP,dd7ddc04e1b6c2c614352b383efe2d36,SP
2,000229ec398224ef6ca0657da4fc703e,5,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,1,1.0,199.00,17.87,216.87,3050.0,...,2018-02-05,-13,Chegou antes do prazo previsto e o produto sur...,2018-01-23,2018-01-23 16:06:31,0,3818d81c6709e39d06b2738a8d3a2474,MG,5b51032eddd242adc84c38acab88f23d,MG
3,00024acbcdf0a6daa1e931b038114c75,4,7634da152a4610f1595efa32f14722fc,perfumery,1,1.0,12.99,12.79,25.78,200.0,...,2018-08-20,-5,None,2018-08-15,2018-08-15 16:39:01,0,af861d436cfc08b2c2ddefd0ba074622,SP,9d7a1d34a5052409006425275ba1c2b4,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,5,ac6c3623068f30de03045865e4e10089,garden_tools,1,1.0,199.90,18.14,218.04,3750.0,...,2017-03-17,-15,Gostei pois veio no prazo determinado .,2017-03-02,2017-03-03 10:54:59,1,64b576fb70d441e8f1b2d7d446e483c5,SP,df560393f3a51e74553ab94004ba5c87,PR


In [5]:
# Close the DuckDB connection
con.close()

[🠉 Back to top](#structure-of-the-notebook)

## **Post-Merge Cleaning**

### Initial Inspection of Joined Dataset

In [ ]:
# Display the summary of the DataFrame
u.summarize_table_shapes(df_orders)

,name,rows,columns,column_names,duplicates
0,df,94782,36,"[order_id, review_score, product_id, product_c...",0


In [ ]:
# Quick overview of column properties (dtypes, missing values, uniques)
u.column_overview(df_orders)

Df:


,dtype,non_null,missing_n,missing_%,uniques_n,uniques
order_id,object,94782,0,0.000000,94782,"[00010242fe8c5a6d1ba2dd792cb16214, 00018f77f2f..."
review_score,int64,94782,0,0.000000,5,"[5, 4, 1, 3, 2]"
product_id,object,94782,0,0.000000,29848,"[4244733e06e7ecb4970a6e2683c13e61, e5f2d52b802..."
product_category_name,object,94782,0,0.000000,75,"[cool_stuff, pet_shop, furniture_decor, perfum..."
num_unique_products,int64,94782,0,0.000000,8,"[1, 2, 4, 3, 5, 6, 7, 8]"
num_items,float64,94782,0,0.000000,17,"[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 8.0, 7.0, 20.0,..."
total_price,float64,94782,0,0.000000,7536,"[58.9, 239.9, 199.0, 12.99, 199.9, 21.9, 19.9,..."
total_freight,float64,94782,0,0.000000,7843,"[13.29, 19.93, 17.87, 12.79, 18.14, 12.69, 11...."
total_amount,float64,94782,0,0.000000,30322,"[72.19, 259.83, 216.87, 25.78, 218.04000000000..."
total_order_weight,float64,93471,1311,1.383174,2827,"[650.0, 30000.0, 3050.0, 200.0, 3750.0, 450.0,..."


----------------------------------------------------------------------------------------------------------------------------------
